# LLM-Powered Metrics Query Processor 
A Python application that converts natural language queries about company performance metrics into structured JSON format using the Groq LLM API.

## 1. Initial Setup

### 1.1 Required Dependencies
Run the following commands in your terminal to install required packages:
```bash
pip install groq python-dateutil
```

### 1.2 Groq API Setup
1. Visit [Groq Console](https://console.groq.com/)
2. Sign up/Login and navigate to API Keys section
3. Generate a new API key
4. Replace `groq_api_key` in Cell 1 with your actual API key

## 2. Running the Application

### Step 1: Run All Cells in Order
Run each cell in sequence:
- Cell 1: Imports and Setup
- Cell 2: Date Handling Functions
- Cell 3: Prompt Template
- Cell 4: Main Processing Function
- Cell 5: JSON Conversion Function
- Cell 6: Main Application Class
- Cell 7: Example Usage and Testing
- Cell 8: Interactive Query Interface

### Step 2: Start Interactive Interface
After running all cells, the interactive interface will start automatically. You'll see:
```
Welcome to the Metrics Query Processor!
==================================================

Example queries:
1. Get me Flipkart's GMV for the last one year
2. Compare Amazon and Walmart's revenue from last quarter
3. Show me Meta's profit between January 2023 and March 2024

Type 'exit' to quit, 'clear' to clear conversation history
==================================================
```

### Step 3: Using the Interactive Interface

#### Available Commands:
- Type your query and press Enter to process it
- Type 'exit' to quit the application
- Type 'clear' to reset conversation history

#### Example Interaction:
```
Enter your query:
> Get me Flipkart's GMV for last quarter

Processing query...

Result:
[
  {
    "entity": "Flipkart",
    "parameter": "GMV",
    "startDate": "2024-02-02",
    "endDate": "2024-05-02"
  }
]
```

## 3. Important Notes
- Ensure all cells are executed in order
- Wait for each cell to complete before running the next
- Make sure your Groq API key is valid
- Maintain active internet connection
- The interface will remember conversation context for up to 6 messages
- Use 'clear' command if you want to start a fresh conversation

## 4. Troubleshooting
If you encounter issues:
1. Check if all cells were executed in order
2. Verify your API key is correctly set
3. Ensure all dependencies are installed
4. Try clearing the conversation history
5. Restart the kernel and run all cells again if needed


In [1]:
# Cell 1: Imports and Setup

import os
import json
from datetime import datetime, timedelta
import groq
from typing import List, Dict
from dateutil.relativedelta import relativedelta
from dateutil import parser

# Set up Groq API key
os.environ["GROQ_API_KEY"] = "groq_api_key"
client = groq.Groq()

# Initialize conversation history
conversation_history = []
max_history_length = 6

In [2]:
# Cell 2: Date Handling Functions

def get_default_dates() -> tuple:
    """Returns default start and end dates"""
    end_date = datetime.now()
    start_date = end_date - relativedelta(years=1)
    return start_date.date(), end_date.date()

def convert_to_iso_date(date_str: str) -> str:
    """Convert various date formats to ISO format"""
    try:
        # Handle relative date terms
        relative_terms = {
            'last quarter': lambda: datetime.now() - relativedelta(months=3),
            'previous quarter': lambda: datetime.now() - relativedelta(months=3),
            'previous month': lambda: datetime.now() - relativedelta(months=1),
            'last month': lambda: datetime.now() - relativedelta(months=1),
            'last year': lambda: datetime.now() - relativedelta(years=1),
            'last 1 year': lambda: datetime.now() - relativedelta(years=1),
            '1 year ago': lambda: datetime.now() - relativedelta(years=1),
            'last week': lambda: datetime.now() - timedelta(days=7),
            'previous week': lambda: datetime.now() - timedelta(days=7),
            'yesterday': lambda: datetime.now() - timedelta(days=1),
        }

        if date_str.lower() in relative_terms:
            return relative_terms[date_str.lower()]().strftime('%Y-%m-%d')
        
        # Parse the date string
        parsed_date = parser.parse(date_str)
        return parsed_date.strftime('%Y-%m-%d')
    except:
        return None

In [3]:
test_dates = [
        "last quarter",
        "2023-01-01",
        "last month",
        "yesterday",
        "june, 2023",
        "2022 May",
        "2022 May 1",
        "last 1 year"
    ]
    
print("Date conversion test:")
for date in test_dates:
    converted = convert_to_iso_date(date)
    print(f"Input: {date} -> Output: {converted}")

Date conversion test:
Input: last quarter -> Output: 2024-10-03
Input: 2023-01-01 -> Output: 2023-01-01
Input: last month -> Output: 2024-12-03
Input: yesterday -> Output: 2025-01-02
Input: june, 2023 -> Output: 2023-06-03
Input: 2022 May -> Output: 2022-05-03
Input: 2022 May 1 -> Output: 2022-05-01
Input: last 1 year -> Output: 2024-01-03


In [4]:
# Cell 3: Prompt Template

def create_prompt(user_query: str, conversation_history: List[Dict]) -> str:
    prompt = """You are an AI assistant specialized in extracting company performance metrics information from user queries. 
    
    Your task is to extract the following information:
    1. Entity (company names)
    2. Parameter (performance metric)
    3. Start Date (if mentioned)
    4. End Date (if mentioned)

    Important Rules and Guidelines:
    
    1. Company Names (Entity):
    - Handle spelling mistakes, common variations and abbreviations
    - Examples: Meta/Facebook, AMZN/Amazon, MSFT/Microsoft
    - Always normalize to the standard company name
    
    2. Performance Metrics (Parameter):
    - Handle spelling mistakes, common variations and abbreviations
    - Examples:
        * GMV/Gross Merchandise Value
        * Rev/Revenue
        * Pft/Profit
    
    3. Date Handling:
    - Only include dates if explicitly mentioned in the query, otherwise ignore. 
    - Handle various time durations and extract start_date and end_date from them. 
    
    4. Comparison Queries:
    - When comparison is requested, include all relevant companies
    - Maintain the same parameter across compared companies
    
    5. Response Format:
    Respond only in a structured format like this:
    {
        "entities": ["company1", "company2"], 
        "parameter": "metric_name",           
        "start_date": "date1",               // include only if mentioned
        "end_date": "date2"                  // include only if mentioned
    }
    Dont include anything else in the response.

    Use Previous conversation context to maintain continuity
    Previous conversation context:
    """
    
    # Add conversation history
    for msg in conversation_history[-max_history_length:]:
        prompt += f"\nUser: {msg['user']}\nAssistant: {msg['assistant']}\n"
    
    # Add current query
    prompt += f"\nCurrent user query: {user_query}"
    
    return prompt

In [5]:
# Cell 4: Main Processing Function

def process_query(user_query: str) -> Dict:
    """Process user query and extract relevant information"""
    try:
        # Update conversation history
        if len(conversation_history) > max_history_length:
            conversation_history.pop(0)
        
        # Create prompt with context
        prompt = create_prompt(user_query, conversation_history)
        
        # Call Groq API
        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "system",
                    "content": "You are a specialized AI assistant for extracting company metrics information."
                },
                {
                    "role": "user",
                    "content": prompt
                }
            ],
            model="llama-3.1-8b-instant",
            temperature=0,
            max_tokens=200
        )
        
        # Extract response
        response = chat_completion.choices[0].message.content
        
        # Parse response
        extracted_info = json.loads(response)

        # Validate required fields
        if not extracted_info.get('entities') or not extracted_info.get('parameter'):
            raise ValueError("Missing required fields in LLM response")
        
        # Update conversation history
        conversation_history.append({
            "user": user_query,
            "assistant": response
        })
        
        return extracted_info
    
    except Exception as e:
        raise Exception(f"Error processing query: {str(e)}")

In [6]:
# Cell 5: JSON Conversion Function

def format_response(extracted_info: Dict) -> List[Dict]:
    """Convert extracted information into JSON format"""
    try:
        # Default dates if not specified
        default_start, default_end = get_default_dates()
        
        formatted_response = []
        
        for entity in extracted_info['entities']:
            response_obj = {
                "entity": entity,
                "parameter": extracted_info['parameter'],
                "startDate": (
                    convert_to_iso_date(extracted_info.get('start_date')) 
                    or default_start.isoformat()
                ),
                "endDate": (
                    convert_to_iso_date(extracted_info.get('end_date')) 
                    or default_end.isoformat()
                )
            }
            formatted_response.append(response_obj)
            
        return formatted_response
    
    except Exception as e:
        raise Exception(f"Error formatting response: {str(e)}")

In [7]:
# Cell 6: Main Application Class

class QueryProcessor:
    def __init__(self):
        self.conversation_history = []
    
    def process_and_format_query(self, user_query: str) -> List[Dict]:
        """Main function to process query and return formatted response"""
        try:
            # Process the query
            extracted_info = process_query(user_query)
            
            # Format the response
            formatted_response = format_response(extracted_info)
            
            return formatted_response
            
        except Exception as e:
            print(f"Error: {str(e)}")
            return [{
                "error": "Failed to process query",
                "details": str(e)
            }]
    
    def clear_history(self):
        """Clear conversation history"""
        self.conversation_history = []

In [8]:
# Cell 7: Example Usage and Testing

def test_application():
    # Initialize processor
    processor = QueryProcessor()
    
    # Test cases
    test_queries = [
        "Get me Flipkart's GMV for the last 1 year",
        "Compare this with Amazon",
        "Show me Meta's revenue from last quarter",
        "What was Apple's profit between 1 January 2023 and 31 March 2024",
        "What was Flipkrt's profit",
        "Compare Amazon and Walmart's profit for last quarter"
    ]
    
    print("Testing Application...")
    print("-" * 50)
    
    for query in test_queries:
        print(f"\nUser Query: {query}")
        result = processor.process_and_format_query(query)
        print(f"Response: {json.dumps(result, indent=2)}")
        print("-" * 50)

# Run the test
test_application()

Testing Application...
--------------------------------------------------

User Query: Get me Flipkart's GMV for the last 1 year
Response: [
  {
    "entity": "Flipkart",
    "parameter": "Gross Merchandise Value",
    "startDate": "2024-01-03",
    "endDate": "2025-01-03"
  }
]
--------------------------------------------------

User Query: Compare this with Amazon
Response: [
  {
    "entity": "Flipkart",
    "parameter": "Gross Merchandise Value",
    "startDate": "2024-01-03",
    "endDate": "2025-01-03"
  },
  {
    "entity": "Amazon",
    "parameter": "Gross Merchandise Value",
    "startDate": "2024-01-03",
    "endDate": "2025-01-03"
  }
]
--------------------------------------------------

User Query: Show me Meta's revenue from last quarter
Response: [
  {
    "entity": "Meta",
    "parameter": "Revenue",
    "startDate": "2024-10-03",
    "endDate": "2025-01-03"
  }
]
--------------------------------------------------

User Query: What was Apple's profit between 1 January 20

In [9]:
# Cell 8: Interactive Query Interface

def main():
    """Interactive interface for real-time metric queries"""
    
    print("Welcome to the Metrics Query Processor!")
    print("=" * 50)
    print("\nExample queries:")
    print("1. Get me Flipkart's GMV for the last one year")
    print("2. Compare Amazon and Walmart's revenue from last quarter")
    print("3. Show me Meta's profit between January 2023 and March 2024")
    print("\nType 'exit' to quit, 'clear' to clear conversation history")
    print("=" * 50)
    
    # Initialize processor
    processor = QueryProcessor()
    
    while True:
        try:
            # Get user input
            print("\nEnter your query:")
            query = input("> ").strip()
            print(query)
            
            # Check for exit command
            if query.lower() == 'exit':
                print("\nThank you for using Metrics Query Processor!")
                break
                
            # Check for clear command
            if query.lower() == 'clear':
                processor.clear_history()
                print("\nConversation history cleared!")
                continue
                
            # Check for empty query
            if not query:
                print("\nPlease enter a valid query!")
                continue
            
            # Process query
            print("\nProcessing query...")
            result = processor.process_and_format_query(query)
            
            # Check for errors
            if 'error' in result[0]:
                print("\nError:", result[0]['details'])
                continue
            
            # Print formatted result
            print("\nResult:")
            print(json.dumps(result, indent=2))
            
        except Exception as e:
            print(f"\nAn error occurred: {str(e)}")
            print("Please try again!")
            
        print("\n" + "=" * 50)

# Run the interactive interface
if __name__ == "__main__":
    main()

Welcome to the Metrics Query Processor!

Example queries:
1. Get me Flipkart's GMV for the last one year
2. Compare Amazon and Walmart's revenue from last quarter
3. Show me Meta's profit between January 2023 and March 2024

Type 'exit' to quit, 'clear' to clear conversation history

Enter your query:
What was Flipkrt's profit

Processing query...

Result:
[
  {
    "entity": "Flipkart",
    "parameter": "Profit",
    "startDate": "2024-01-03",
    "endDate": "2025-01-03"
  }
]


Enter your query:
exit

Thank you for using Metrics Query Processor!
